# **Generative AI intelligent controller orchestrator**
copyright 2025, Denis Rothman

This notebook demonstrates the implementation of a centralized AI controller orchestrator to manage AI agents for various tasks such as sentiment analysis and semantic analysis.

We will not provide the orchestrator with an explicit task tag. The goal of the notebook is to use GPT-4o's semantic text similarity capacity to find the best scenario containing instructions to perform a task itself based on the user input.

**Table of Contents**

1. **Environment Setup**  
   Installation and API key configuration for OpenAI.
2. **Scenario Definitions**    
Predefined agent behaviors and task-specific workflows.
3. **AI Controller Logic**   
Mechanisms for intent recognition and agent selection.
4. **Generative AI Agent Integration**  
 Dynamic task selection via OpenAI's GPT API.
5. **Agent Task Execution**   
   Semantic matching and vector similarity for task alignment.
6. **Additional AI Services**   
   Web search integration and retrieval-augmented generation (RAG).
7. **Execution Workflow**   
   End-to-end examples of agent interaction and task processing.


# Setting up the environment

This notebook was developed in Google Colab. Colab includes many pre-installed libraries and sets `/content/` as the default directory, meaning you can access files directly by their filename if you wish (e.g., `filename` instead of needing to specify `/content/filename`). This differs from local environments, where you'll often need to install libraries or specify full file paths.

## File downloading script

grequests contains a script to download files from the repository

In [ ]:
!curl -L https://raw.githubusercontent.com/uCertifyLLC/Building-Business-Ready-Generative-AI-Systems/main/commons/grequests.py --output grequests.py

## Installing OpenAI

In [ ]:
from grequests import download
download("commons","requirements01.py")
download("commons","openai_setup.py")
download("commons","openai_api.py")

Downloaded 'requirements01.py' successfully.
Downloaded 'openai_setup.py' successfully.
Downloaded 'openai_api.py' successfully.


In [ ]:
# Run the setup script to install and import dependencies
%run requirements01

Uninstalling 'openai'...
Installing 'openai' version 1.57.1...
'openai' version 1.57.1 is installed.


#### Initializing the OpenAI API key



In [ ]:
google_secrets=True #activates Google secrets in Google Colab
if google_secrets==True:
  import openai_setup
  openai_setup.initialize_openai_api()

OpenAI API key initialized successfully.


In [ ]:
if google_secrets==False: # Uncomment the code and choose any method you wish to initialize the API_KEY
  import os
  #API_KEY=[YOUR API_KEY]
  #os.environ['OPENAI_API_KEY'] = API_KEY
  #openai.api_key = os.getenv("OPENAI_API_KEY")
  #print("OpenAI API key initialized successfully.")

#### Importing the API call function

In [ ]:
# Import the function from the custom OpenAI API file
import openai_api
from openai_api import make_openai_api_call

# Prompt (user input or system input)

In [ ]:
prompt=1
if prompt==1:
  input = "Gladiator II is a great movie although I didn't like some of the scenes. I liked the actors though. Overall I really enjoyed the experience."
if prompt==2:
  input = "Generative AI models such as GPT-4o can be built into Generative AI Systems. Provide more information."

# Scenario Library (repository)

Intent recognition

In [ ]:
scenarios = [
    {
        "scenario_number": 1,
        "description": "Market Semantic analysis.You will be provided with a market survey on a give range of products.The term market must be in the user or system input. Your task is provide an analysis."
    },

    {
        "scenario_number": 2,
        "description": " Sentiment analysis  Read the content and classify the content as an opinion  If it is not opinion, stop there  If it is an opinion then your task is to perform a sentiment analysis on these statements and provide a score with the label: Analysis score: followed by a numerical value between 0 and 1  with no + or - sign.Add an explanation."
    },
    {
        "scenario_number": 3,
        "description": "Semantic analysis.This is not an analysis but a semantic search. Provide more information on the topic."
    }
]

In [ ]:
# Original list of dictionaries
scenario_instructions = [
    {
        "Market Semantic analysis.You will be provided with a market survey on a give range of products.The term market must be in the user or system input. Your task is provide an analysis."
    },
    {
        "Sentiment analysis  Read the content return a sentiment analysis on this text and provide a score with the label named : Sentiment analysis score followed by a numerical value between 0 and 1  with no + or - sign and  add an explanation to justify the score."
    },
    {
        "Semantic analysis.This is not an analysis but a semantic search. Provide more information on the topic."
    }
]

In [ ]:
# Extract the strings from each dictionary
instructions_as_strings = [list(entry)[0] for entry in scenario_instructions]

# Intent Recognition and Scenario Selection

## Generative AI selection

In [ ]:
# Define the parameters for the function call
mrole = "system"
mcontent = "You are an assistant that matches user inputs to predefined scenarios. Select the scenario that best matches the input. Respond with the scenario_number only."
user_role = "user"

# Adjust `input` to combine user input with scenarios
selection_input = f"User input: {input}\nScenarios: {scenarios}"
print(selection_input)

# Call the function using your standard API call
response = openai_api.make_openai_api_call(selection_input, mrole, mcontent, user_role)

User input: Gladiator II is a great movie although I didn't like some of the scenes. I liked the actors though. Overall I really enjoyed the experience.
Scenarios: [{'scenario_number': 1, 'description': 'Market Semantic analysis.You will be provided with a market survey on a give range of products.The term market must be in the user or system input. Your task is provide an analysis.'}, {'scenario_number': 2, 'description': ' Sentiment analysis  Read the content and classify the content as an opinion  If it is not opinion, stop there  If it is an opinion then your task is to perform a sentiment analysis on these statements and provide a score with the label: Analysis score: followed by a numerical value between 0 and 1  with no + or - sign.Add an explanation.'}, {'scenario_number': 3, 'description': 'Semantic analysis.This is not an analysis but a semantic search. Provide more information on the topic.'}]


In [ ]:
# Print the response
print("Scenario:",response )

Scenario: 2


In [ ]:
response

'2'

In [ ]:
scenario_number=int(response)
instructions=scenario_instructions[scenario_number-1]
print(instructions)

{'Sentiment analysis  Read the content return a sentiment analysis nalysis on this text and provide a score with the label named : Sentiment analysis score followed by a numerical value between 0 and 1  with no + or - sign and  add an explanation to justify the score.'}


In [ ]:
print(input)

Gladiator II is a great movie although I didn't like some of the scenes. I liked the actors though. Overall I really enjoyed the experience.


# Generative AI Agent

### API Call

In [ ]:
print(input)

Gladiator II is a great movie although I didn't like some of the scenes. I liked the actors though. Overall I really enjoyed the experience.


In [ ]:
# Accessing by line number (1-based index)
line_number = scenario_number
instruction = instructions_as_strings[line_number - 1]  # Adjusting for 0-based indexing
print(f"Instruction on line {line_number}:\n{instruction}")
mrole = "system"
user_role = "user"
mcontent = instruction

Instruction on line 2:
Sentiment analysis  Read the content return a sentiment analysis nalysis on this text and provide a score with the label named : Sentiment analysis score followed by a numerical value between 0 and 1  with no + or - sign and  add an explanation to justify the score.


In [ ]:
# API function call
sc_input=instruction +" "+ input
print(sc_input)
task_response = openai_api.make_openai_api_call(sc_input,mrole,mcontent,user_role)

Sentiment analysis  Read the content return a sentiment analysis nalysis on this text and provide a score with the label named : Sentiment analysis score followed by a numerical value between 0 and 1  with no + or - sign and  add an explanation to justify the score. Gladiator II is a great movie although I didn't like some of the scenes. I liked the actors though. Overall I really enjoyed the experience.


In [ ]:
print(task_response)

Sentiment analysis score 0.75

The text expresses a generally positive sentiment towards the movie "Gladiator II." The use of words like "great movie," "liked the actors," and "really enjoyed the experience" indicates a favorable opinion. However, the mention of not liking some of the scenes introduces a slight negative element. Despite this, the overall enjoyment and positive remarks about the actors and the movie as a whole outweigh the negative aspect, resulting in a sentiment score leaning towards the positive side.
